In [1]:
import pandas as pd
from piex import explorer

In [2]:
piex = explorer.PipelineExplorer('ml-pipelines-2018')

In [3]:
df = piex.get_pipelines(data_modality='single_table', task_type='classification')

In [4]:
def get_scores(df):
    stats = df.groupby('dataset')['rank'].agg(['mean', 'std', 'min', 'max'])
    stats['range'] = stats['max'] - stats['min']
    stats['midpoint'] = stats['min'] + stats['range'] / 2
    
    df = df.merge(stats, how='left', on='dataset')
    df['t-score'] = (df['rank'] - df['mean']) / df['std']
    df['r-score'] = (df['rank'] - df['midpoint']) / df['range']
    df['ranking'] = df.groupby('dataset')['rank'].rank(method='dense')
    
    return df

scores = get_scores(df)

In [5]:
def rank_pipelines(df, metric='t-score'):
    scores = get_scores(df)
    pipeline_scores = scores.groupby('pipeline')[['t-score', 'r-score', 'ranking']].mean()
    return pipeline_scores.sort_values(metric)

In [6]:
rank_pipelines(df)

,t-score,r-score,ranking
pipeline,,,
categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.591566,-0.401695,2116.436421
dfs/categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.582055,-0.387389,2317.254773
categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.649732,-0.010138,3899.292544
dfs/categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.752098,0.016891,4134.603802
trivial.mode,1.449710,0.254141,4125.979701


In [7]:
def split_datasets(df):
    datasets = pd.Series(df.dataset.unique())
    datasets = datasets.sample(len(datasets))
    half = int(len(datasets) / 2)
    
    first_half = df[df.dataset.isin(datasets[:half])]
    second_half = df[df.dataset.isin(datasets[half:])]
    
    return first_half, second_half

first_half, second_half = split_datasets(df)

In [8]:
rank_pipelines(first_half)

,t-score,r-score,ranking
pipeline,,,
dfs/categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.602553,-0.405301,2261.893139
categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.586371,-0.414857,2066.091206
categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.657787,-0.013661,3878.995681
dfs/categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.776937,0.017640,3991.294521
trivial.mode,1.488414,0.267669,4005.091880


In [9]:
rank_pipelines(second_half)

,t-score,r-score,ranking
pipeline,,,
categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.596554,-0.389060,2164.768958
dfs/categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.563074,-0.370802,2368.518964
categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.642178,-0.006835,3918.324307
dfs/categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.730540,0.016241,4258.981571
trivial.mode,1.411006,0.240614,4246.867521
